In [62]:
#model
import numpy as np
import pandas as pd
from keras import layers
from keras.layers import Input,Dense,Activation,ZeroPadding2D,BatchNormalization,Flatten,Conv2D
from keras.layers import AveragePooling2D,MaxPooling2D,Dropout,GlobalMaxPooling2D,GlobalAveragePooling2D
from keras.utils import np_utils
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import keras.backend as K

In [63]:
data=pd.read_csv("data.csv")
dataset=np.array(data)
np.random.shuffle(dataset)
X=dataset
Y=dataset
X=X[:, 0:1024]
Y=Y[:, 1024]

X_train=X[0:70000, :]
X_train=X_train / 255.                     #it is a noramlization which will convert to (0/1)
X_test=X[70000:72001, :]
X_test=X_test / 255.                       #it is a noramlization which will convert to (0/1)

#reshaping dataset
Y=Y.reshape(Y.shape[0],1)
Y_train=Y[0:70000, :]
Y_train=Y_train.T
Y_test=Y[70000:72001, :]
Y_test=Y_test.T

In [64]:
print("Number of training examples = "+str(X_train.shape[0]))
print("Number of test examples = "+str(X_test.shape[0]))
print("X_train shape: "+str(X_train.shape))
print("X_test shape: "+str(X_test.shape))
print("Y_train shape: "+str(Y_train.shape))
print("Y_test shape: "+str(Y_test.shape))

Number of training examples = 70000
Number of test examples = 2000
X_train shape: (70000, 1024)
X_test shape: (2000, 1024)
Y_train shape: (1, 70000)
Y_test shape: (1, 2000)


In [67]:
image_x=32
image_y=32

train_y=np_utils.to_categorical(Y_train)
test_y=np_utils.to_categorical(Y_test)
train_y=train_y.reshape(train_y.shape[1],train_y.shape[2])
test_y=test_y.reshape(test_y.shape[1],test_y.shape[2])
X_train=X_train.reshape(X_train.shape[0],image_x,image_y,1)
X_test=X_test.reshape(X_test.shape[0],image_x,image_y,1)

In [68]:
print("X_train shape: "+str(X_train.shape))
print("Y_train shape: "+str(train_y.shape))

X_train shape: (70000, 32, 32, 1)
Y_train shape: (70000, 37)


In [72]:
#building a model
def keras_model(image_x,image_y):
    num_of_classes=37
    model=Sequential()
    model.add(Conv2D(filters=32,kernel_size=(5,5),input_shape=(image_x,image_y,1),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same'))
    model.add(Conv2D(64,(5,5),activation='relu'))
    model.add(MaxPooling2D(pool_size=(5,5),strides=(5,5),padding='same'))
    model.add(Flatten())
    model.add(Dense(num_of_classes,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    filepath="devanagari_model_refined.h5"
    checkpoint1=ModelCheckpoint(filepath,monitor='val_acc',verbose=1,save_best_only=True,mode='max')
    callbacks_list=[checkpoint1]
    
    return model, callbacks_list

In [73]:
model, callbacks_list= keras_model(image_x, image_y)
model.fit(X_train, train_y, validation_data=(X_test, test_y), epochs=8,batch_size=64,callbacks=callbacks_list)
scores=model.evaluate(X_test,test_y,verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1] * 100))
model.save('devanagri.h5')

Epoch 1/8
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmp7uohgdjq.py, line 48)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmp7uohgdjq.py, line 48)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1094/1094 [==============================] - 78s 52ms/step - loss: 1.4450 - accuracy: 0.6052 - val_loss: 0.2769 - val_accuracy: 0.9230
Epoch 2/8
1094/1094 [==============================] - 55s 50ms/step - loss: 0.2508 - accuracy: 0.9238 - val_loss: 0.1767 - val_accuracy: 0.9460
Epoch 3/8
1094/1094 [==============================] - 55s 50ms/step - loss: 0.1571 - accuracy: 0.9516 - v